In [ ]:
import psana as ps
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from scipy.ndimage.filters import gaussian_filter
from scipy.optimize import curve_fit
import matplotlib.colors as colors

In [ ]:
run = 40
#exp = "tmolw0518"
exp = "tmolw0618"

ds = ps.DataSource(exp=exp, run=run)
run = next(ds.runs())

In [ ]:
timing = run.Detector("timing")
hsd = run.Detector("hsd")


In [ ]:
Nfind = 1000
Nfound = 0

#xenergies = np.empty_like(positions)
times = None

chan = 0

for nevent, event in enumerate(run.events()):
    
    if nevent < 100: continue
    
    hsd_data = hsd.raw.waveforms(event)
    
    if hsd_data is None: continue
    
    if times is None:
        times = hsd_data[chan]['times'] * 1e6
        wfs = np.empty((Nfind, len(times)))
    wfs[Nfound] = hsd_data[chan][0]
    
    Nfound += 1
    if Nfound == Nfind: break

In [ ]:
plt.plot(times[::4], wfs[0][::4])
#plt.plot(times[1::4],  wfs[0][1::4])
#plt.plot(times, wfs[0], 'k', alpha=0.6)
plt.title('Run 41, 400 eV')
plt.xlim(1, 8)
plt.ylim(2020, 2100)

In [ ]:
plt.plot(times[::4], wfs[0][::4])
#plt.plot(times[1::4],  wfs[0][1::4])
#plt.plot(times, wfs[0], 'k', alpha=0.6)
plt.title('392 eV')
plt.xlim(1, 8)
plt.ylim(2020, 2100)

In [ ]:
wfs_mean = wfs.mean(0)
wfs_mean -= wfs_mean[times < 0.15].mean()
wfs_mean = gaussian_filter(wfs_mean, 30)
plt.figure(figsize=(18,3))
plt.plot(times, wfs_mean)
plt.xlim(0, 7)
plt.xticks(np.arange(0, 8.01, 0.2));
plt.grid()
pks_idx = np.where((wfs_mean[:-2] < wfs_mean[1:-1]) & (wfs_mean[1:-1] > wfs_mean[2:]) & (wfs_mean[1:-1] > 2.0) & (times[1:-1] > 1.3))[0] + 1
#pks_idx = pks_idx[:-1]
#pks_idx = np.concatenate((pks_idx[:2], pks_idx[6:7]))
t_pks = times[pks_idx]
plt.plot(t_pks, wfs_mean[pks_idx], 'rx')
plt.xlabel("Time of flight / us")

In [ ]:
def ToF(m_q, c, t0):
    return t0 + c * np.sqrt(m_q)
m = 20.0
q0s = [1, 2, 3, 4]
if 1==0: # manual peaks
    t_pks = np.array([4.582, 3.30, 2.732])[::-1]
else: # automated mode
    t_pks = times[pks_idx]
dq = np.arange(len(t_pks))
best = 9999999999999999
plt.figure(dpi=70)
for q0_ in q0s:
    qs_ = q0_ + dq
    m_qs = m / qs_
    c_, t0_ = curve_fit(ToF, m_qs, t_pks[::-1], p0=[1,1])[0]
    resid = ((ToF(m_qs, c_, t0_) - t_pks[::-1])**2).mean()
    if resid < best:
        c, t0 = c_, t0_
        best = resid
        q0 = q0_
        qs = qs_
    plt.plot(qs_, ToF(m_qs, c_, t0_) - t_pks[::-1], '.-', label=q0_)
plt.legend()
plt.xlabel("Charge #")
plt.ylabel("Residual")
plt.grid()
plt.title("Best: q0=%d" % q0)
m_qs = ((times - t0) / c)**2
print("c: %.5f, t0: %.5f" % (c, t0))
for qi in np.arange(1, 9):
    print("q=%d, t=%.5f" % (qi, ToF(m / qi, c, t0)))

In [ ]:
plt.figure(dpi=80, figsize=(15,3))
plt.plot(m_qs, wfs_mean * 2, 'k', label='avg')
plt.xlim(0, 30)
plt.ylim(-1.5, 1.5)
plt.xticks(np.arange(0, 30.1, 1));
for qi in range(1, 8):
    #ti = ToF(m / qi, c, t0)
    plt.plot([m/qi, m/qi], [-1.5, 1.5], 'r--')
    plt.text(m/qi, 1.5, '%d+' % qi, ha='center', va='bottom')
plt.grid(alpha=0.2)
plt.xlabel("M / Q")